In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from bs4 import BeautifulSoup
import re
import json
import requests
import openpyxl
import urllib
from urllib import parse
import lxml
from sklearn.model_selection import train_test_split

from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

import time
from sklearn.preprocessing import MinMaxScaler

In [2]:
def normalize_scale(inp):# 환율데이터 표본적어서 *10
    result = []
    for i in inp:
        result.append(((i-np.mean(inp))/np.var(inp))*10)
    return result
def normalize(inp):# *10안함
    result = []
    for i in inp:
        result.append(((i-np.mean(inp))/np.var(inp)))
    return result

class Stock:
    
    Exchange='./Data/Exchange.csv' #객체가 공통으로 사용할 환율 데이터 (동적크롤링할줄몰라서 csv로 받아옴)
    
    def __init__(self,code):
        self.code=code
    
    @staticmethod
    def exchangeRate():  ##1달간격의 환율의 종가데이터  1행배열 57요소( 18.05.01~  23.01.01) 배열로 출력 SIZE(1,57)
        csv = pd.read_csv(Stock.Exchange,encoding='utf8')
        #print(csv)  환율 전체데이터 확인하기            
        csv = np.array(csv)
        data=[]
        for i in range(len(csv)):   
            fnum=""
            for j in csv[i][1]:
                if(j == ','):
                    continue
                fnum+=j
            data.append(float(fnum))
        data = np.array(data)
        data = data.astype('float32')
        data = normalize_scale(data) 
        return data

    def idxNorm(self):##ROA, ROE, EPS, BPS, DPS, PER, PBR의 각 항목당 (2018.12 ~ 2023.12)의 데이터 6개 갖음 SIZE(7,8)
        get_param = {
            'pGB':1,
            'gicode':'A%s'%(self.code),
            'cID':'',
            'MenuYn':'Y',
            'ReportGB':'',
            'NewMenuID':101,
            'stkGb':701,
        }
        get_param = parse.urlencode(get_param)
        url="http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?%s"%(get_param)
        tables = pd.read_html(url, header=0,encoding='utf-8')
        sit=np.array(tables[11]) 

        data=[]#정규화 시작

        for i in range(7):
            crw=[]
            if (i == 4): continue
            for j in range(7):
                
                if(sit[17+i][j+1] != sit[17+i][j+1]):
                    idx =  (float(sit[17+i][j]) + float(sit[17+i][j+2]))/2.0
                    crw.append(idx)
                    continue
                idx = float(sit[17+i][j+1])
                crw.append(idx)
            data.append(normalize(crw))
        data=np.array(data)
        return data
    
    def priceList(self):
        csv = pd.read_csv('./Data/주식가격/%s.csv'%(self.code))
        cut_csv = csv[:][:1148]
        arr = np.array(cut_csv)
        array = arr[:,1]
        array = array/10000
        array = np.array(array)
        close = array.reshape(-1,1)
        return close

    def showChart(self): ##종목코드의 제무재표를 나타내는 함수
        get_param = {
            'pGB':1,
            'gicode':'A%s'%(self.code),
            'cID':'',
            'MenuYn':'Y',
            'ReportGB':'',
            'NewMenuID':101,
            'stkGb':701,
        }
        get_param = parse.urlencode(get_param)
        url="http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?%s"%(get_param)
        tables = pd.read_html(url, header=0,encoding='utf-8')
        return tables[11]

In [3]:
def concatData(a, data):
    a2023 = a[:-1147]
    a2022 = a[-1147:-901]
    a2021 = a[-901:-653]
    a2020 = a[-653:-405]
    a2019 = a[-405:-159]
    a2018 = a[-159:len(a)]
    
    data2023 = np.full((len(a2023),6), data[:,5])
    data2022 = np.full((len(a2022),6), data[:,4])
    data2021 = np.full((len(a2021),6), data[:,3])
    data2020 = np.full((len(a2020),6), data[:,2])
    data2019 = np.full((len(a2019),6), data[:,1])
    data2018 = np.full((len(a2018),6), data[:,0])
    
    data1 = data2023
    data1 = np.concatenate((np.full((len(a2023),6), data[:,5]),
                            np.full((len(a2022),6), data[:,4]),
                            np.full((len(a2021),6), data[:,3]),
                            np.full((len(a2020),6), data[:,2]),
                            np.full((len(a2019),6), data[:,1]),
                            np.full((len(a2018),6), data[:,0])), axis=0)
    arr = np.concatenate((a, data1), axis=1)
    array = np.array(arr)
    
    er = Stock.exchangeRate()
    er = np.array(er).reshape(-1, 1)
    
    exchange = np.full((1,1), er[-57])
    exchange = np.concatenate((exchange, np.full((21,1), er[-56]),
                               np.full((22,1), er[-55]),
                               np.full((19,1), er[-54]),
                               np.full((20,1), er[-53]),
                               np.full((22,1), er[-52]),
                               np.full((21,1), er[-51]),
                               np.full((20,1), er[-50]),
                               np.full((21,1), er[-49]),
                               np.full((21,1), er[-48]),
                               np.full((21,1), er[-47]),
                               np.full((18,1), er[-46]),
                               np.full((20,1), er[-45]),
                               np.full((22,1), er[-44]),
                               np.full((22,1), er[-43]),
                               np.full((19,1), er[-42]),
                               np.full((19,1), er[-41]),
                               np.full((21,1), er[-40]),
                               np.full((22,1), er[-39]),
                               np.full((22,1), er[-38]),
                               np.full((19,1), er[-37]),
                               np.full((22,1), er[-36]),
                               np.full((22,1), er[-35]),
                               np.full((18,1), er[-34]),
                               np.full((20,1), er[-33]),
                               np.full((21,1), er[-32]),
                               np.full((21,1), er[-31]),
                               np.full((19,1), er[-30]),
                               np.full((21,1), er[-29]),
                               np.full((20,1), er[-28]),
                               np.full((23,1), er[-27]),
                               np.full((22,1), er[-26]),
                               np.full((19,1), er[-25]),
                               np.full((20,1), er[-24]),
                               np.full((22,1), er[-23]),
                               np.full((20,1), er[-22]),
                               np.full((20,1), er[-21]),
                               np.full((20,1), er[-20]),
                               np.full((21,1), er[-19]),
                               np.full((21,1), er[-18]),
                               np.full((19,1), er[-17]),
                               np.full((21,1), er[-16]),
                               np.full((23,1), er[-15]),
                               np.full((19,1), er[-14]),
                               np.full((21,1), er[-13]),
                               np.full((22,1), er[-12]),
                               np.full((20,1), er[-11]),
                               np.full((17,1), er[-10]),
                               np.full((22,1), er[-9]),
                               np.full((19,1), er[-8]),
                               np.full((22,1), er[-7]),
                               np.full((21,1), er[-6]),
                               np.full((17,1), er[-5]),
                               np.full((22,1), er[-4]),
                               np.full((22,1), er[-3]),
                               np.full((19,1), er[-2]),
                               np.full((17,1), er[-1])), axis=0)
    train_inputs = np.concatenate((array, exchange), axis=1)
    return train_inputs
def returnInputs(a,b):
    inputs = concatData(a, b)
    results = inputs[:-1,:].astype(float)
    return results
def returnLabels(a,b):
    inputs = concatData(a, b)
    results = inputs[1:,0].reshape(-1,1).astype(float)
    return results
def reverseData(arr):
    temp = [[0] * len(arr[0]) for _ in range(len(arr))]
    for i in range(len(arr)):
        temp[i] = arr[len(arr)-1-i]
    return temp
def scaleData(a):
    scaler = MinMaxScaler()
    scale_cols = [0]
    df_scaled = scaler.fit_transform(a[scale_cols])

In [4]:
SS = Stock('005930')
SK = Stock('000660')
DW=Stock('049770')
SFA=Stock('056190')
GB=Stock('024110')

sa=SS.priceList()
sk=SK.priceList()
dw=DW.priceList()
sf=SFA.priceList()
gb=GB.priceList()
sadata=SS.idxNorm()
skdata=SK.idxNorm()
dwdata=DW.idxNorm()
sfdata=SFA.idxNorm()
gbdata=GB.idxNorm()

C:\Users\wjddu\AppData\Local\Temp\ipykernel_2780\4271975719.py:70: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  csv = pd.read_csv('./Data/주식가격/%s.csv'%(self.code))
C:\Users\wjddu\AppData\Local\Temp\ipykernel_2780\4271975719.py:70: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  csv = pd.read_csv('./Data/주식가격/%s.csv'%(self.code))


In [5]:
train_inputs = []
train_labels = []
train_inputs.append(reverseData(returnInputs(sa, sadata)))
train_inputs.append(reverseData(returnInputs(sk, skdata)))
train_inputs.append(reverseData(returnInputs(dw, dwdata)))
train_inputs.append(reverseData(returnInputs(sf, sfdata)))
train_inputs.append(reverseData(returnInputs(gb, gbdata)))
train_inputs = np.array(train_inputs)

train_labels.append(reverseData(returnLabels(sa, sadata)))
train_labels.append(reverseData(returnLabels(sk, skdata)))
train_labels.append(reverseData(returnLabels(dw, dwdata)))
train_labels.append(reverseData(returnLabels(sf, sfdata)))
train_labels.append(reverseData(returnLabels(gb, gbdata)))
train_labels = np.array(train_labels)

In [11]:
model = Sequential([
    layers.LSTM(512, input_shape=(train_inputs.shape[1],train_inputs.shape[2]), return_sequences=True),
    layers.LSTM(128),
    layers.Dense(1, activation = 'relu')
])

model.compile(loss='mse',optimizer='adam')
model.summary()
model.fit(train_inputs, train_labels,
    epochs=100
)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 1147, 512)         1067008   
                                                                 
 lstm_7 (LSTM)               (None, 128)               328192    
                                                                 
 dense_4 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,395,329
Trainable params: 1,395,329
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
1/1 [==============================] - 5s 5s/step - loss: 122.9470
Epoch 2/100
1/1 [==============================] - 2s 2s/step - loss: 94.2252
Epoch 3/100
1/1 [==============================] - 2s 2s/step - loss: 67.0679
Epoch 4/100
1/1 [==============================] - 2s 2s/step - loss: 53.

1/1 [==============================] - 2s 2s/step - loss: 9.1197
Epoch 95/100
1/1 [==============================] - 2s 2s/step - loss: 8.9216
Epoch 96/100
1/1 [==============================] - 2s 2s/step - loss: 8.7343
Epoch 97/100
1/1 [==============================] - 2s 2s/step - loss: 8.5550
Epoch 98/100
1/1 [==============================] - 2s 2s/step - loss: 8.3900
Epoch 99/100
1/1 [==============================] - 2s 2s/step - loss: 8.2304
Epoch 100/100
1/1 [==============================] - 2s 2s/step - loss: 8.0817


In [13]:
model.save('stockmodel.h5')

In [12]:
a=[]
a.append(reverseData(returnInputs(sa, sadata)))
a = np.array(a)
print(a)
model.predict(a)

[[[ 5.09        0.45111764  0.31059243 ... -0.2270406   2.01693153
   -0.18861003]
  [ 5.16        0.45111764  0.31059243 ... -0.2270406   2.01693153
   -0.18861003]
  [ 5.13        0.45111764  0.31059243 ... -0.2270406   2.01693153
   -0.18861003]
  ...
  [ 5.66        0.05481708  0.03894379 ... -0.04130336  2.40506956
    0.13651005]
  [ 5.53        0.05481708  0.03894379 ... -0.04130336  2.40506956
    0.13651005]
  [ 5.55       -0.14176057 -0.09841311 ...  0.0499988  -0.6515174
    0.15273633]]]
1/1 [==============================] - 1s 929ms/step


array([[5.622607]], dtype=float32)